In [67]:
# default_exp gensim_lda

In [68]:
from dynamic_topic_modeling.sklearn_lda import *

In [12]:
affirmative = make_df("affirmative.csv")

In [13]:
affirmative.head()

,Group,Students,Content,text
0,第1组,正三,慕课将分布于世界各地的最优质的教育资源聚集到一起，让任何有学习愿望的人能够低成本的，通常是免...,慕课 将 分布 于 世界各地 的 最 优质 的 教育资源 聚集 到 一起 ， 让 任何 有 ...
1,第1组,正二,"在慕课发展过程中的现阶段，中国最大的慕课平台icourse163的用户人数突破100万,与其...",在 慕课 发展 过程 中 的 现阶段 ， 中国 最大 的 慕课 平台 icourse163 ...
2,第1组,正一,"研究发现,在慕课融入的课堂学习中,学习者情感体验丰富,知识技能以及元认知能力得到提升,思想观...","研究 发现 , 在 慕课 融入 的 课堂 学习 中 , 学习者 情感 体验 丰富 , 知识 ..."
3,第1组,正三,慕课在保证教育质量的同时，降低提供教育的成本，给社会带来的憧憬。任何人任何时候再任何地方，都...,慕课 在 保证 教育 质量 的 同时 ， 降低 提供 教育 的 成本 ， 给 社会 带来 的...
4,第1组,正一,对方反一辩友也说是可能出现的欢快气氛，传统课堂集体聆听教师单方面赐予的知识，这难道不是一种容...,对方 反一 辩友 也 说 是 可能 出现 的 欢快 气氛 ， 传统 课堂 集体 聆听 教师 ...


In [14]:
stopwords = get_custom_stopwords("stopwords.txt", encoding='utf-8') # HIT停用词词典
max_df = 0.9 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 5 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。
n_features = 1000 # 最大提取特征数量
n_top_words = 20 # 显示主题下关键词的时候，显示多少个
col_content = "text" # 说明其中的文本信息所在列名称

In [54]:
raw_documents = affirmative['text'].tolist()

In [61]:
# 参考 https://blog.csdn.net/kwame211/article/details/78963517
import jieba
docs = [[word for word in jieba.cut(document, cut_all=True)] for document in raw_documents]

In [62]:
# 参考 https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-auto-examples-tutorials-run-lda-py
from gensim.corpora import Dictionary
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 5 documents, or more than 90% of the documents.
dictionary.filter_extremes(no_below=5, no_above=0.9)

```python
TypeError: doc2bow expects an array of unicode tokens on input, not a single string
```

- [x] list 化 https://blog.csdn.net/cg_amaz1ng/article/details/79567583

In [63]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [64]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 1060
Number of documents: 617


In [65]:
from gensim.models import LdaModel

# Set training parameters.
num_topics = 8
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [66]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -1.3974.
[([(0.05549783, '学生'),
   (0.025053011, '学习'),
   (0.019431483, '可以'),
   (0.019155778, '教学'),
   (0.01799905, '是'),
   (0.016881756, '视频'),
   (0.016856788, '教师'),
   (0.016813617, '在'),
   (0.015622923, '课'),
   (0.014619939, '慕'),
   (0.011750538, '课堂'),
   (0.011127644, '和'),
   (0.011125633, '进行'),
   (0.0106243985, '了'),
   (0.010584355, '老师'),
   (0.010449255, '过程'),
   (0.009150563, '中'),
   (0.009006159, '知识'),
   (0.008953948, '上'),
   (0.008841038, '也')],
  -1.144416048284077),
 ([(0.0453565, '课'),
   (0.044192884, '慕'),
   (0.035201326, '学习'),
   (0.019053701, '教学'),
   (0.015633227, '和'),
   (0.013481987, '可以'),
   (0.012789655, '课程'),
   (0.012127374, '是'),
   (0.012039293, '在'),
   (0.00975174, '传统'),
   (0.0095419055, '课堂'),
   (0.009485807, '学生'),
   (0.009090529, '中'),
   (0.008726664, '有'),
   (0.008528268, '了'),
   (0.0075090057, '模式'),
   (0.0069556953, '学习者'),
   (0.0067211697, '进行'),
   (0.006115625, '更'),
   (0.005775242, '对')],

In [70]:
import pickle as pkl

In [73]:
with open("model/gensim-lda.pkl", 'wb') as fp:
    pkl.dump(model, fp)

In [74]:
with open("model/gensim-lda.pkl", 'rb') as fp:
    model0 = pkl.load(fp)

In [75]:
model0.__class__

gensim.models.ldamodel.LdaModel